In [1117]:
import numpy as np
import pandas as pd
from cylib.apis.all_api import *
from cylib.qmtdata.cyxtdata import xtdata
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm  #ols线性回归
import sys
import seaborn as sns
# 将指定路径添加到模块搜索路径中
sys.path.append('/home/wangb/晨乐多因子/因子储备')
# 导入stock_test模块
import stock_test as stk_tst

1.在因子后面标注ok即为增量和全量测试均通过  
2.在因子后面标注no1即为:在"全量计算测试，测试连续的20天"中，容易出现"ValueError("返回值行数有误"),如（1223成交额波动系数，0729对数流通市值，0805日内收益率）  
(注意，数据来源不是读表，而是来源于xtdata.get_market_data_ex的数据容易出现“返回值行数有误”错误)
3.标注为no1的在去除检测中的（            
if len(df) < 1:
	#print(len(df))
	Raise ValueError("返回值行数有误")   
）可以成功被检测  
4.有些时间取不到具体的trade_data或者数据极少，就取了就近的m_timetag(如：1022十大股东占比分散度TOP10stocks，0926市场杠杆  Market_leverage)，不然会造成返回值行数有误，因为有空值None  
5.有时候测试可能不稳定，有些测试失败了之后，过一会测试会成功（可能有些随机事件数据缺失）  
6.因子没有做详细说明因为电脑MarkDown不太支持公式####  

In [1118]:
start_date = '20150101' #设置全量计算时的开始时间
trade_date = '20240814' #每天的日期

In [1119]:
#导库
hs300_list = list(xtdata.get_index_weight('000300.SH').keys())

#financial_data
financial_data_Income = xtdata.get_financial_data(hs300_list, ["Balance", "Income"])
financial_data_TOP10FLOWHOLDER = xtdata.get_financial_data(hs300_list, ["TOP10FLOWHOLDER"])
financial_data_ASHAREBALANCESHEET = xtdata.get_financial_data(hs300_list, ["ASHAREBALANCESHEET"])
financial_data_ASHAREINCOME = xtdata.get_financial_data(hs300_list, ["ASHAREINCOME"])
financial_data_PERSHAREINDEX = xtdata.get_financial_data(hs300_list, ["PERSHAREINDEX"])
financial_data_ASHAREBALANCESHEET = xtdata.get_financial_data(hs300_list, ["ASHAREBALANCESHEET"])
financial_data_ASHARECASHFLOW = xtdata.get_financial_data(hs300_list, ["ASHARECASHFLOW"])

#stdata
stdata = xtdata.get_market_data_ex(
    field_list=['open', 'high', 'low', 'close', 'amount', 'volume'], 
    stock_list=hs300_list, period='1d',
    start_time=start_date, end_time=trade_date, count=-1,
    dividend_type='front', fill_data=True
)
data = pd.DataFrame()
for i in stdata.keys():
    temp = stdata[i]
    temp['ts_code'] = i
    data = data.append(temp)
data = data.reset_index()
data = data.rename(columns={'index': 'trade_date'})
data.set_index('trade_date', inplace=True)
data.index = pd.to_datetime(data.index)
data = data[data['amount'] != 0]

#total_mv
total_mv = get_price(ts_code_list=hs300_list, feature_list=['total_mv'],
                        start_date=start_date, trade_date=trade_date, back_len=1, target_type='stock')
total_mv = total_mv.sort_index(ascending=True)
total_mv = total_mv.reset_index(level='trade_date')

#t日最高价
stdata_high = xtdata.get_market_data_ex(
    field_list=['high'], stock_list=hs300_list, period='1d',
    start_time=start_date, end_time=trade_date, count=-1,
    dividend_type='front_ratio', fill_data=True
)
stocks = list(stdata_high.keys())
dates = list(stdata_high[stocks[0]].index)
data_array = np.zeros((len(dates), len(stocks)))
for i, stock in enumerate(stocks):
    data_array[:, i] = stdata_high[stock]['high'].values
df_upst = pd.DataFrame(data_array, index=[str(date) for date in dates], columns=stocks)

#t日最低价
stdata_low = xtdata.get_market_data_ex(
    field_list=['low'], stock_list=hs300_list, period='1d',
    start_time=start_date, end_time=trade_date, count=-1,
    dividend_type='front_ratio', fill_data=True
)
stocks = list(stdata_low.keys())
dates = list(stdata_low[stocks[0]].index)
data_array = np.zeros((len(dates), len(stocks)))
for i, stock in enumerate(stocks):
    data_array[:, i] = stdata_low[stock]['low'].values
df_downst = pd.DataFrame(data_array, index=[str(date) for date in dates], columns=stocks)

#t-1日收盘价
stdata_pre = xtdata.get_market_data_ex(
    field_list=['preClose'], stock_list=hs300_list, period='1d',
    start_time=start_date, end_time=trade_date, count=-1,
    dividend_type='front_ratio', fill_data=True
)
stocks = list(stdata_pre.keys())
dates = list(stdata_pre[stocks[0]].index)
data_array = np.zeros((len(dates), len(stocks)))
for i, stock in enumerate(stocks):
    data_array[:, i] = stdata_pre[stock]['preClose'].values
df_pClst = pd.DataFrame(data_array, index=[str(date) for date in dates], columns=stocks)

#开盘
stdata_open = xtdata.get_market_data_ex(
    field_list=['open'], stock_list=hs300_list, period='1d',
    start_time=start_date, end_time=trade_date, count=-1,
    dividend_type='front_ratio', fill_data=True
)
stocks = list(stdata_open.keys())
dates = list(stdata_open[stocks[0]].index)
data_array = np.zeros((len(dates), len(stocks)))
for i, stock in enumerate(stocks):
    data_array[:, i] = stdata_open[stock]['open'].values
df_opst = pd.DataFrame(data_array, index=[str(date) for date in dates], columns=stocks)

#收盘
stdata_close = xtdata.get_market_data_ex(
    field_list=['close'], stock_list=hs300_list, period='1d',
    start_time=start_date, end_time=trade_date, count=-1,
    dividend_type='front_ratio', fill_data=True
)
stocks = list(stdata_close.keys())
dates = list(stdata_close[stocks[0]].index)
data_array = np.zeros((len(dates), len(stocks)))
for i, stock in enumerate(stocks):
    data_array[:, i] = stdata_close[stock]['close'].values
df_clst = pd.DataFrame(data_array, index=[str(date) for date in dates], columns=stocks)

#成交额
stdata_amount = xtdata.get_market_data_ex(
    field_list=['amount'], stock_list=hs300_list, period='1d',
    start_time=start_date, end_time=trade_date, count=-1,
    dividend_type='front', fill_data=True
)
stocks = list(stdata_amount.keys())
dates = list(stdata_amount[stocks[0]].index)
data_array = np.zeros((len(dates), len(stocks)))
for i, stock in enumerate(stocks):
    data_array[:, i] = stdata_amount[stock]['amount'].values
df_amount_st = pd.DataFrame(data_array, index=[str(date) for date in dates], columns=stocks)

#pe_ttm
pe_ttm = get_price(ts_code_list = hs300_list,feature_list = ['pe_ttm'],
start_date=start_date,trade_date = trade_date,back_len = 1,target_type='stock')
pe_ttm = pe_ttm.sort_index(ascending=True)



#参数
common_param = {
    'hs300_list': hs300_list,
    'financial_data_Income': financial_data_Income,
    'financial_data_TOP10FLOWHOLDER': financial_data_TOP10FLOWHOLDER,
    'financial_data_ASHAREBALANCESHEET': financial_data_ASHAREBALANCESHEET,
    'financial_data_ASHAREINCOME': financial_data_ASHAREINCOME,
    'financial_data_PERSHAREINDEX': financial_data_PERSHAREINDEX,
    'financial_data_ASHAREBALANCESHEET': financial_data_ASHAREBALANCESHEET,
    'financial_data_ASHARECASHFLOW': financial_data_ASHARECASHFLOW,
    'stdata': stdata,
    'data': data,
    'df_upst':df_upst,
    'df_downst':df_downst,
    'df_pClst':df_pClst,
    'df_opst':df_opst,
    'df_clst':df_clst,
    'df_amount_st':df_amount_st,
    'pe_ttm':pe_ttm,
    'total_mv':total_mv
}

In [1120]:
# 获取财务报表
financial_data = xtdata.get_financial_data(common_param['hs300_list'],["Balance","Income"])
#print(financial_data)
# 以营业利润为例 （oper_profit）
All_data = pd.DataFrame()
for asset in financial_data.keys():
    temp = financial_data[asset]['Income'][['m_timetag','m_anntime','oper_profit']]
    temp['ts_code'] = asset
    temp = temp[['ts_code','m_timetag','m_anntime','oper_profit']]
    temp = temp[temp.m_anntime>='20150101']
    All_data = All_data.append(temp)
    
All_data['Year'] = All_data['m_timetag'].apply(lambda x:x[:4])
All_data['Month'] = All_data['m_timetag'].apply(lambda x:x[4:6])
All_data.head(5)

,ts_code,m_timetag,m_anntime,oper_profit,Year,Month
71,000001.SZ,20141231,20150313,2.624600e+10,2014,12
72,000001.SZ,20150331,20150424,7.447000e+09,2015,03
73,000001.SZ,20150630,20150814,1.526700e+10,2015,06
74,000001.SZ,20150930,20151023,2.330500e+10,2015,09
75,000001.SZ,20151231,20160310,2.889500e+10,2015,12


In [1121]:
def cal_factor_dif(data,factor):    
    def cal_group_dif(group,factor):
        res = group[factor].diff()
        res = res.fillna(group[factor])
        return res
    data[f'{factor}_diff'] = data.groupby('Year').apply(lambda x:cal_group_dif(x,factor)).reset_index(level=0, drop=True)
    return data

In [1122]:
def cal_factor_ttm(data,factor):
    """
    计算factor_ttm.
    factor_diff ：单季度factor数值
    factor_ttm ：滚动四个季度factor之和
    """
    
    def cal_group_dif(group,factor):
        res = group[factor].diff()
        res = res.fillna(group[factor])
        return res
    data[f'{factor}_diff'] = data.groupby('Year').apply(lambda x:cal_group_dif(x,factor)).reset_index(level=0, drop=True)
    data[f'{factor}_TTM'] = data[f'{factor}_diff'].rolling(4).sum()
    return data

All_data = All_data.groupby('ts_code').apply(lambda x:cal_factor_ttm(x,'oper_profit'))
All_data['m_timetag'] = pd.to_datetime(All_data['m_timetag'])
All_data['m_anntime'] = pd.to_datetime(All_data['m_anntime'])
pd.set_option('display.max_rows', 5)
All_data

,ts_code,m_timetag,m_anntime,oper_profit,Year,Month,oper_profit_diff,oper_profit_TTM
71,000001.SZ,2014-12-31,2015-03-13,2.624600e+10,2014,12,2.624600e+10,NaN
72,000001.SZ,2015-03-31,2015-04-24,7.447000e+09,2015,03,7.447000e+09,NaN
...,...,...,...,...,...,...,...,...
22,688981.SH,2024-03-31,2024-05-10,5.281070e+08,2024,03,5.281070e+08,5.537094e+09
23,688981.SH,2024-06-30,2024-08-30,1.823658e+09,2024,06,1.295551e+09,4.771914e+09


# 脚本框架：

In [ ]:
def DIY_FACTOR1_SCRIPT(trade_date,common_param,increment=True,start_date = '20150101'):
    if increment==True:
        NEW_DATA_DF = calc_new_factor()
        #你的因子增量计算代码
        return NEW_DATA_DF #返回trade_date这天新增的因子数据
    else:
        ALL_DATA_DF = calc_all_factor()
        #你的因子全量计算代码
        return ALL_DATA_DF #返回截止到trade_date所有的因子数据
    #返回一个df给数据库，有三列，分别是：trade_date、ts_code、factor_name

In [ ]:
temp.to_csv('result.csv', index=False)

1025成交额ok

In [1127]:
def mean_amount(trade_date, common_param,increment=True,start_date = '20150101'):
    temp_data = common_param['data'][['amount', 'ts_code']]
    if increment: 
        end_date = trade_date
    else: 
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(years=1)).strftime("%Y%m%d")
    temp_data = temp_data[(temp_data.index >= new_start_date) & (temp_data.index <= trade_date)]
    
    mean_amount = temp_data.copy()
    mean_amount['mean_amount'] = mean_amount.groupby('ts_code')['amount'].transform(lambda x: x.rolling(242).mean())
    mean_amount = mean_amount.dropna()
    mean_amount = mean_amount.reset_index()
    return mean_amount[['trade_date','ts_code', 'mean_amount']]

In [108]:
result = mean_amount(trade_date,common_param,increment=True,start_date = '20150101')
result

,trade_date,ts_code,mean_amount
0,2024-08-12,000001.SZ,1.207848e+09
1,2024-08-13,000001.SZ,1.205703e+09
...,...,...,...
892,2024-08-13,688981.SH,1.285457e+09
893,2024-08-14,688981.SH,1.285569e+09


1022十大股东占比分散度okkk

In [1128]:
def TOP10stocks(trade_date, common_param,increment=True,start_date = '20150101'):
    All_data1022 = pd.DataFrame()
    for asset in common_param['financial_data_TOP10FLOWHOLDER'].keys():
        temp = (common_param['financial_data_TOP10FLOWHOLDER'])[asset]['TOP10FLOWHOLDER'][['declareDate', 'endDate', 'ratio']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'declareDate', 'endDate', 'ratio']]
        All_data1022 = All_data1022.append(temp)
    if increment:
        end_date = trade_date
    else:
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=3)).strftime("%Y%m%d")  # pct + 3
    All_data1022['endDate'] = pd.to_datetime(All_data1022['endDate'])
    All_data1022 = All_data1022[(All_data1022['endDate'] >= new_start_date) & (All_data1022['endDate'] <= trade_date)]
    
    std_devs = All_data1022.groupby(['ts_code', 'endDate'])['ratio'].std().reset_index()
    #std_devs.columns = ['ts_code', 'declareDate', 'TOP10stocks']
    std_devs = std_devs.dropna()
    std_devs = std_devs.rename(columns={"ratio": "TOP10stocks"})
    std_devs = std_devs.rename(columns={"endDate": "trade_date"})
    TOP10stocks = std_devs.reset_index(drop=True)
    #TOP10stocks=TOP10stocks.groupby('ts_code').tail(1)
    return TOP10stocks[['trade_date','ts_code', 'TOP10stocks']]  # 返回的DataFrame有三列：trade_date、ts_code、factor_name

In [1162]:
start_date = '20150101' #设置全量计算时的开始时间
trade_date = '20240814' #每天的日期

In [854]:
result = TOP10stocks(trade_date, common_param, increment=True ,start_date = '20150101')
result

,trade_date,ts_code,TOP10stocks
0,2024-05-16,000425.SZ,1.649006
1,2024-06-17,002601.SZ,7.618856
...,...,...,...
8,2024-05-31,605117.SH,12.980664
9,2024-06-25,688599.SH,5.669895


1019存货增长率ok

In [1164]:
def inventory_growth_rate(trade_date, common_param,increment=True,start_date = '20150101'):
    All_data1019 = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'inventories']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 'inventories']]
        All_data1019 = All_data1019.append(temp)
    All_data1019 = All_data1019.dropna()
    
    if increment: 
        end_date = trade_date
    else: 
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=15)).strftime("%Y%m%d")
    All_data1019['m_timetag'] = pd.to_datetime(temp['m_timetag'])
    All_data1019 = All_data1019[(All_data1019['m_timetag'] >= new_start_date) & (All_data1019['m_timetag'] <= trade_date)]
    
    All_data1019['inventory_growth_rate'] = (
        (All_data1019['inventories'] - All_data1019.groupby('ts_code')['inventories'].transform(lambda x:x.pct_change(4))) / All_data1019.groupby('ts_code')['inventories'].transform(lambda x:x.pct_change(4))
    )
    inventory_growth_rate = All_data1019[['m_timetag', 'ts_code', 'inventory_growth_rate']]
    inventory_growth_rate.columns = ['trade_date', 'ts_code', 'inventory_growth_rate']
    inventory_growth_rate = inventory_growth_rate.dropna()
    return inventory_growth_rate

In [1158]:
result = inventory_growth_rate(trade_date, common_param,increment=False,start_date = '20150101')
result

,trade_date,ts_code,inventory_growth_rate
10,2021-12-31,000001.SZ,-5.700249e+09
11,2022-03-31,000001.SZ,-5.134601e+10
...,...,...,...
19,2024-03-31,688981.SH,5.648908e+10
20,2024-06-30,688981.SH,1.275093e+11


1010未预期税费支出ok

In [1131]:
def Unexpected_tax_expenses(trade_date, common_param,increment=True,start_date = '20150101'):
    All_data1010 = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREINCOME'].keys():
        temp = common_param['financial_data_ASHAREINCOME'][asset]['ASHAREINCOME'][['m_anntime', 'm_timetag', 'inc_tax']]
        temp['ts_code'] = asset
        All_data1010 = All_data1010.append(temp)
        
    if increment: 
        end_date = trade_date
    else: 
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=18)).strftime("%Y%m%d")#diff+3,shift+3
    All_data1010['m_timetag'] = pd.to_datetime(temp['m_timetag'])
    All_data1010 = All_data1010[(All_data1010['m_timetag'] >= new_start_date) & (All_data1010['m_timetag'] <= trade_date)]
    
    All_data1010['Year'] = All_data1010['m_timetag'].apply(lambda x: x.year)
    All_data1010['inc_tax_diff'] = All_data1010.groupby('ts_code')['inc_tax'].diff()
    
    All_data1010['Unexpected_tax_expenses'] = All_data1010.groupby('ts_code')['inc_tax_diff'].apply(lambda x: (x - x.shift(4)) / x.shift(4))
    
    Unexpected_tax_expenses = All_data1010[['m_timetag', 'ts_code', 'Unexpected_tax_expenses']]
    Unexpected_tax_expenses.columns = ['trade_date', 'ts_code', 'Unexpected_tax_expenses']
    Unexpected_tax_expenses = Unexpected_tax_expenses.dropna()
    return Unexpected_tax_expenses

In [666]:
result = Unexpected_tax_expenses(trade_date, common_param,increment=True,start_date = '20150101')
result

,trade_date,ts_code,Unexpected_tax_expenses
23,2024-06-30,000001.SZ,-0.348063
23,2024-06-30,000002.SZ,1.647917
...,...,...,...
23,2024-06-30,688396.SH,-10.167819
23,2024-06-30,688981.SH,-0.533204


1001单季度总资产净利率变动ok

In [1132]:
def Net_profit_margin_change(trade_date, common_param,increment=True,start_date = '20150101'):
    All_data1001 = pd.DataFrame()
    for asset in common_param['financial_data_PERSHAREINDEX'].keys():
        temp = common_param['financial_data_PERSHAREINDEX'][asset]['PERSHAREINDEX'][['m_anntime', 'm_timetag', 'net_profit']]
        temp['ts_code'] = asset
        All_data1001 = All_data1001.append(temp)
        
    if increment: 
        end_date = trade_date
    else: 
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=15)).strftime("%Y%m%d")#pct+3
    All_data1001['m_timetag'] = pd.to_datetime(temp['m_timetag'])
    All_data1001 = All_data1001[(All_data1001['m_timetag'] >= new_start_date) & (All_data1001['m_timetag'] <= trade_date)]
    
    All_data1001['change'] = All_data1001['net_profit'] - All_data1001.groupby('ts_code')['net_profit'].transform(lambda x:x.pct_change(4))
    
    Net_profit_margin_change = All_data1001[['m_timetag', 'ts_code', 'change']]
    Net_profit_margin_change.columns = ['trade_date', 'ts_code', 'Net_profit_margin_change']
    Net_profit_margin_change = Net_profit_margin_change.dropna()
    return Net_profit_margin_change

In [669]:
result = Net_profit_margin_change(trade_date, common_param,increment=True,start_date = '20150101')
result

,trade_date,ts_code,Net_profit_margin_change
35,2024-06-30,000001.SZ,33.276913
35,2024-06-30,000002.SZ,12.153279
...,...,...,...
35,2024-06-30,688599.SH,3.799684
35,2024-06-30,688981.SH,6.887972


0926市场杠杆okkk

In [1133]:
def Market_leverage(trade_date, common_param,increment=True,start_date = '20150101'):
    All_data = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'tot_assets']]
        temp['ts_code'] = asset
        All_data = All_data.append(temp)

    if increment:
        end_date = trade_date
    else:
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=2)).strftime("%Y%m%d")
    All_data['m_timetag'] = pd.to_datetime(All_data['m_timetag'])
    All_data = All_data[(All_data['m_timetag'] >= new_start_date) & (All_data['m_timetag'] <= trade_date)]
    All_data['m_anntime'] = pd.to_datetime(All_data['m_anntime'])


    tot = common_param['total_mv']
    tot['trade_date'] = pd.to_datetime(tot['trade_date'])
    
    merged_df = pd.merge(tot, All_data, how='outer', left_on=['ts_code', 'trade_date'], right_on=['ts_code', 'm_anntime'])
    
    merged_df['tot_assets'] = merged_df.groupby('ts_code')['tot_assets'].apply(lambda x: x.ffill())
    
    merged_df['Market_leverage'] = merged_df['tot_assets'] / merged_df['total_mv']
    
    Market_leverage = merged_df[['m_timetag', 'ts_code', 'Market_leverage']]
    Market_leverage.columns = ['trade_date', 'ts_code', 'Market_leverage']
    Market_leverage = Market_leverage.dropna()
    #Market_leverage=Market_leverage.groupby('ts_code').tail(1)
    return Market_leverage


In [871]:
result = Market_leverage(trade_date, common_param,increment=True,start_date = '20150101')
result

,trade_date,ts_code,Market_leverage
293603,2020-03-31,002007.SZ,1152.571280
294179,2020-03-31,002916.SZ,1631.456540
...,...,...,...
298155,2020-03-31,688009.SH,14735.623762
298159,2020-03-31,688126.SH,2325.457698


0922基于振幅切割的动量ok

In [ ]:
#无调参导致3个

In [1134]:
def Momentum_based_on_amplitude_cutting(trade_date, common_param,increment=True,start_date = '20150101'):
    # 获取t日最高价数据
    df_upst = common_param['df_upst']
    
    # 获取t日最低价数据
    df_downst = common_param['df_downst']
    
    # 获取t-1日收盘价数据
    df_pClst = common_param['df_pClst']
    
    # 计算振幅
    ans = (df_upst - df_downst) / df_pClst

    if increment: 
        end_date = trade_date
    else: 
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=8)).strftime("%Y%m%d")#pct+3
    ans.index = pd.to_datetime(ans.index)
    ans = ans[(ans.index >= new_start_date) & (ans.index <= trade_date)]
    
    # 计算因子
    ans = ans.rolling(160).apply(lambda x: np.sort(x)[:len(x) // 2].sum(), raw=True)
    
    ans = ans.stack().reset_index()
    ans.columns = ['trade_date', 'ts_code', 'Momentum_based_on_amplitude_cutting']
    Momentum_based_on_amplitude_cutting = ans.sort_values(by=['ts_code', 'trade_date'])
    Momentum_based_on_amplitude_cutting = Momentum_based_on_amplitude_cutting.dropna()
    if increment: 
        Momentum_based_on_amplitude_cutting = Momentum_based_on_amplitude_cutting.groupby('ts_code').tail(1)
    return Momentum_based_on_amplitude_cutting

In [687]:
result = Momentum_based_on_amplitude_cutting(trade_date, common_param,increment=True,start_date = '20150101')
result

,trade_date,ts_code,Momentum_based_on_amplitude_cutting
600,2024-08-14,000001.SZ,0.987273
601,2024-08-14,000002.SZ,1.759316
...,...,...,...
898,2024-08-14,688599.SH,2.117461
899,2024-08-14,688981.SH,1.516543


0902单季度总资产报酬率变动ok

In [1135]:
def Return_on_total_assets_change(trade_date, common_param, increment=True, start_date='20150101'):
    # 获取利润总额和利息支出数据
    All_data_list = []
    for asset in common_param['financial_data_ASHAREINCOME'].keys(): 
        temp = common_param['financial_data_ASHAREINCOME'][asset]['ASHAREINCOME'][['m_anntime', 'm_timetag', 'tot_profit', 'less_int_exp']].copy()
        temp['ts_code'] = asset
        All_data_list.append(temp)
        
    All_data = pd.concat(All_data_list, ignore_index=True)
    All_data.fillna(0, inplace=True)
    # 确定增量计算的结束日期
    if increment:
        end_date = trade_date
    else:
        end_date = start_date
        
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=15)).strftime("%Y%m%d")  # pct + 3
    All_data['m_timetag'] = pd.to_datetime(All_data['m_timetag'])
    
    # 过滤时间范围内的数据
    All_data = All_data[(All_data['m_timetag'] >= new_start_date) & (All_data['m_timetag'] <= trade_date)]
    
    # 添加年份信息
    All_data['Year'] = All_data['m_timetag'].dt.year
    
    # 计算利润差异（tot_profit_diff 和 less_int_exp_diff）
    All_data = All_data.groupby('ts_code').apply(lambda x: cal_factor_dif(x, 'tot_profit')).reset_index(drop=True)
    All_data = All_data.groupby('ts_code').apply(lambda x: cal_factor_dif(x, 'less_int_exp')).reset_index(drop=True)
    
    # 获取总资产数据
    All_data1_list = []
    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'tot_assets']].copy()
        temp['ts_code'] = asset
        All_data1_list.append(temp)
    
    All_data1 = pd.concat(All_data1_list, ignore_index=True)
    
    # 计算平均总资产
    All_data1['Avg'] = All_data1.groupby('ts_code')['tot_assets'].apply(lambda x: (x + x.shift(4)) / 2)
    All_data1['m_timetag'] = pd.to_datetime(All_data1['m_timetag'])
    
    # 内连接利润总额和总资产数据
    merged_df = pd.merge(All_data, All_data1, how='inner', on=['ts_code', 'm_anntime', 'm_timetag'])
    
    # 计算总资产报酬率
    merged_df['Return_on_total_assets'] = (merged_df['less_int_exp_diff'] + merged_df['tot_profit_diff']) / merged_df['Avg']
    
    # 计算总资产报酬率的变化（同比变化）
    merged_df['Return_on_total_assets_change'] = merged_df.groupby('ts_code')['Return_on_total_assets'].apply(lambda x: x - x.shift(4))
    
    # 准备返回的数据
    Return_on_total_assets_change = merged_df[['m_timetag', 'ts_code', 'Return_on_total_assets_change']].copy()
    Return_on_total_assets_change.columns = ['trade_date', 'ts_code', 'Return_on_total_assets_change']
    Return_on_total_assets_change = Return_on_total_assets_change.dropna()
    return Return_on_total_assets_change


In [1124]:
result = Return_on_total_assets_change(trade_date, common_param,increment=False,start_date = '20150101')
result

,trade_date,ts_code,Return_on_total_assets_change
4,2014-12-31,000001.SZ,-0.008786
5,2015-03-31,000001.SZ,-0.000048
...,...,...,...
9459,2024-03-31,688981.SH,-0.005312
9460,2024-06-30,688981.SH,-0.002969


0828营业能力改善ok

In [1136]:
# 定义N个季度
N = 8
def Improvement_of_business_performance(trade_date, common_param, increment=True, start_date='20150101'):
    def get_last_n_values(group, N):
        return group.tail(N)

    All_data0828 = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREINCOME'].keys():
        temp = common_param['financial_data_ASHAREINCOME'][asset]['ASHAREINCOME'][['m_anntime', 'm_timetag', 'revenue_inc', 'total_expense']]
        temp['ts_code'] = asset
        temp = temp[['ts_code','m_anntime','m_timetag','revenue_inc','total_expense']]
        All_data0828 = All_data0828.append(temp)

    All_data0828['Year'] = All_data0828['m_timetag'].apply(lambda x: x[:4])
    All_data0828 = All_data0828.groupby('ts_code').apply(lambda x: cal_factor_dif(x, 'revenue_inc'))
    All_data0828 = All_data0828.groupby('ts_code').apply(lambda x: cal_factor_dif(x, 'total_expense'))
    
    # 获取最近N季度的净利润率变化和总支出变化
    All_data0828 = All_data0828.groupby('ts_code').apply(get_last_n_values, N=N)
    All_data0828 = All_data0828.drop(All_data0828.columns[0], axis=1)
    
    # Z-score标准化
    scaled_data = pd.DataFrame()
    scaler = StandardScaler()
    
    for name, group in All_data0828.groupby('ts_code'):
        scaler.fit(group[['revenue_inc_diff', 'total_expense_diff']])
        group[['revenue_inc_diff', 'total_expense_diff']] = scaler.transform(group[['revenue_inc_diff', 'total_expense_diff']])
        scaled_data = pd.concat([scaled_data, group])
    
    # 对每组进行线性回归，提取残差
    residuals = []

    for name, group in scaled_data.groupby('ts_code'):
        for i in range(0, len(group), 8):
            sub_group = group.iloc[i:i+8]
            if len(sub_group) < 8:
                continue
            X = sub_group['total_expense_diff']
            y = sub_group['revenue_inc_diff']
            X = sm.add_constant(X)
            model = sm.OLS(y, X).fit()
            residual = model.resid  # 提取残差
            residuals.append(residual)
    
    residuals_df = pd.concat(residuals)
    scaled_data['Improvement_of_business_performance'] = residuals_df
    
    # 准备返回的数据
    scaled_data = scaled_data.reset_index() 
    #scaled_data[0] = scaled_data.index
    Improvement_of_business_performance = scaled_data[['m_timetag', 'ts_code', 'Improvement_of_business_performance']]
    Improvement_of_business_performance.columns = ['trade_date', 'ts_code', 'Improvement_of_business_performance']
    Improvement_of_business_performance = Improvement_of_business_performance.dropna()
    # 确定增量计算的结束日期
    if increment:
        Improvement_of_business_performance = Improvement_of_business_performance.groupby('ts_code').tail(1)##直接取最后
    return Improvement_of_business_performance

In [301]:
result = Improvement_of_business_performance(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Improvement_of_business_performance
7,20240630,000001.SZ,-0.698066
15,20240630,000002.SZ,-0.201703
...,...,...,...
2391,20240630,688599.SH,-0.108446
2399,20240630,688981.SH,0.371731


0826盈利市值比ok

In [1137]:
def Profit_to_market_ratio(trade_date, common_param, increment=True, start_date='20150101'):
    # 获取净利润数据
    All_data = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREINCOME'].keys():
        temp = common_param['financial_data_ASHAREINCOME'][asset]['ASHAREINCOME'][['m_anntime', 'm_timetag', 'net_profit_excl_min_int_inc']]
        temp['ts_code'] = asset
        All_data = All_data.append(temp)
    if increment:
        end_date = trade_date
    else:
        end_date = start_date
        
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=15)).strftime("%Y%m%d")  # pct + 3
    All_data['m_timetag'] = pd.to_datetime(All_data['m_timetag'])
    All_data = All_data[(All_data['m_timetag'] >= new_start_date) & (All_data['m_timetag'] <= trade_date)]
    # 提取年份和月份
    All_data['Year'] = All_data['m_timetag'].dt.year
    All_data['Month'] = All_data['m_timetag'].dt.month
    
    # 计算TTM净利润
    All_data = All_data.groupby('ts_code').apply(lambda x: cal_factor_ttm(x, 'net_profit_excl_min_int_inc'))
    
    # 获取股票市值数据
    stock_price = common_param['total_mv']
    
    # 转换公告日期格式
    All_data['m_anntime'] = pd.to_datetime(All_data['m_anntime'])
    
    # 合并数据集，计算净利润与市值比
    merged_df = pd.merge(All_data, stock_price, how='inner', left_on=['ts_code', 'm_anntime'], right_on=['ts_code', 'trade_date'])
    merged_df['Profit_to_market_ratio'] = merged_df['net_profit_excl_min_int_inc_TTM'] / merged_df['total_mv']
    
    # 删除缺失值
    #merged_df = merged_df.dropna()
    
    # 准备返回的数据
    Profit_to_market_ratio = merged_df[['m_timetag', 'ts_code', 'Profit_to_market_ratio']]
    Profit_to_market_ratio.columns = ['trade_date', 'ts_code', 'Profit_to_market_ratio']
    Profit_to_market_ratio = Profit_to_market_ratio.dropna()
    return Profit_to_market_ratio

In [714]:
result = Profit_to_market_ratio(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Profit_to_market_ratio
5,2024-03-31,000002.SZ,1334.823063
7,2024-03-31,000063.SZ,877.765638
...,...,...,...
922,2024-03-31,688599.SH,1348.242025
925,2024-03-31,688981.SH,155.914840


0822账面市值比okk

In [1138]:
def book_to_market(trade_date, common_param, increment=True, start_date='20150101'):
    All_data1 = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'shortterm_loan', 'long_term_loans', 'bonds_payable', 'notes_payable', 'tot_liab']]
        temp['ts_code'] = asset
        All_data1 = All_data1.append(temp)
    All_data1.fillna(0, inplace=True)
    
    if increment:
        end_date = trade_date
    else:
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=2)).strftime("%Y%m%d")  # pct + 3
    All_data1['m_timetag'] = pd.to_datetime(All_data1['m_timetag'])
    All_data1 = All_data1[(All_data1['m_timetag'] >= new_start_date) & (All_data1['m_timetag'] <= trade_date)]
    All_data1['m_anntime'] = pd.to_datetime(All_data1['m_anntime'])

    # 计算金融负债
    All_data1['Financial_liabilities'] = All_data1['shortterm_loan'] + All_data1['long_term_loans'] + All_data1['bonds_payable'] + All_data1['notes_payable'] + All_data1['tot_liab']
    
    # 获取金融资产相关数据
    All_data2 = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'cash_equivalents', 'tradable_fin_assets', 'account_receivable', 'bill_receivable', 'held_to_mty_invest', 'fin_assets_avail_for_sale', 'tot_assets']]
        temp['ts_code'] = asset
        All_data2 = All_data2.append(temp)
    All_data2.fillna(0, inplace=True)
    All_data2['m_anntime'] = pd.to_datetime(All_data2['m_anntime'])
    All_data2['m_timetag'] = pd.to_datetime(All_data2['m_timetag'])
    # 计算金融资产
    All_data2['Financial_assets'] = All_data2['cash_equivalents'] + All_data2['tradable_fin_assets'] + All_data2['account_receivable'] + All_data2['bill_receivable'] + All_data2['held_to_mty_invest'] + All_data2['fin_assets_avail_for_sale'] + All_data2['tot_assets']
    
    # 获取市值数据
    stock_price = common_param['total_mv']
    
    # 合并负债和资产数据
    merged_df = pd.merge(All_data1, All_data2, how='inner', left_on=['ts_code', 'm_anntime', 'm_timetag'], right_on=['ts_code', 'm_anntime', 'm_timetag'])
    
    # 合并市值数据
    merged_df['m_anntime'] = pd.to_datetime(merged_df['m_anntime'])
    merged_df1 = pd.merge(merged_df, stock_price, how='inner', left_on=['ts_code', 'm_anntime'], right_on=['ts_code', 'trade_date'])
    
    # 计算经营性净资产市值
    merged_df1['Market_value_of_operating_net_assets'] = merged_df1['Financial_liabilities'] - merged_df1['Financial_assets'] + merged_df1['total_mv']
    
    # 获取股东权益数据
    All_data3 = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'tot_liab_shrhldr_eqy', 'tot_liab']]
        temp['ts_code'] = asset
        All_data3 = All_data3.append(temp)
    All_data3.fillna(0, inplace=True)
    All_data3['m_anntime'] = pd.to_datetime(All_data3['m_anntime'])
    All_data3['m_timetag'] = pd.to_datetime(All_data3['m_timetag'])
    # 合并股东权益数据
    All_data3['m_anntime'] = pd.to_datetime(All_data3['m_anntime'])
    merged_df2 = pd.merge(merged_df, All_data3, how='inner', left_on=['ts_code', 'm_anntime', 'm_timetag'], right_on=['ts_code', 'm_anntime', 'm_timetag'])
    
    # 计算经营性净资产
    merged_df2['Operating_net_assets'] = merged_df2['tot_liab_shrhldr_eqy'] - merged_df2['tot_liab_y'] + merged_df2['Financial_liabilities'] - merged_df2['Financial_assets']
    
    # 合并数据并计算账面市值比
    merged_df3 = pd.merge(merged_df1, merged_df2, how='inner', 
                          left_on=['ts_code', 'm_anntime', 'm_timetag', 'shortterm_loan', 'long_term_loans', 'bonds_payable', 'notes_payable', 'Financial_liabilities', 'cash_equivalents', 'account_receivable', 'bill_receivable', 'held_to_mty_invest', 'fin_assets_avail_for_sale', 'tot_assets'],
                          right_on=['ts_code', 'm_anntime', 'm_timetag', 'shortterm_loan', 'long_term_loans', 'bonds_payable', 'notes_payable', 'Financial_liabilities', 'cash_equivalents', 'account_receivable', 'bill_receivable', 'held_to_mty_invest', 'fin_assets_avail_for_sale', 'tot_assets'])
    merged_df3['book_to_market'] = merged_df3['Operating_net_assets'] / merged_df3['Market_value_of_operating_net_assets']
    
    # 准备返回的数据
    book_to_market = merged_df3[['m_timetag', 'ts_code', 'book_to_market']]
    book_to_market.columns = ['trade_date', 'ts_code', 'book_to_market']
    book_to_market = book_to_market.dropna()
    #book_to_market=book_to_market.groupby('ts_code').tail(1)
    return book_to_market

In [883]:
result = book_to_market(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,book_to_market
0,2023-06-30,000001.SZ,0.121315
1,2023-06-30,000002.SZ,-0.573881
...,...,...,...
248,2023-06-30,688396.SH,0.333391
249,2023-06-30,688599.SH,-0.043077


0805日内收益率no1

In [1139]:
def Intraday_rate_of(trade_date, common_param, increment=True, start_date='20150101'):
    # 获取开盘价数据
    df_opst = common_param['df_opst']

    # 获取收盘价数据
    df_clst = common_param['df_clst']

    # 计算日内收益率因子
    ans = (df_clst / df_opst) - 1
    ans = ans.stack().reset_index()
    ans.columns = ['trade_date', 'ts_code', 'Intraday_rate_of']
    ans = ans[['ts_code', 'trade_date', 'Intraday_rate_of']]
    Intraday_rate_of = ans.sort_values(by=['ts_code', 'trade_date'])
    
    Intraday_rate_of['trade_date'] = pd.to_datetime(Intraday_rate_of['trade_date'] )
    if increment:
        Intraday_rate_of = Intraday_rate_of[Intraday_rate_of['trade_date'] == trade_date]
    else:
        Intraday_rate_of = Intraday_rate_of[(Intraday_rate_of['trade_date']  >= start_date) & (Intraday_rate_of['trade_date']  <= trade_date)]
    return Intraday_rate_of

In [894]:
result = Intraday_rate_of(trade_date, common_param, increment=True, start_date='20150101')
result

,ts_code,trade_date,Intraday_rate_of
524069,000001.SZ,2023-05-29,-0.010735
524070,000002.SZ,2023-05-29,-0.009880
...,...,...,...
524366,688599.SH,2023-05-29,-0.123913
524367,688981.SH,2023-05-29,0.006699


0729对数流通市值no1

In [1140]:
def Logarithmic_circulating_market_value(trade_date, common_param, increment=True, start_date='20150101'):
    # 总市值
    stock_price = common_param['total_mv']
    stock_price = stock_price.reset_index()
    # 计算ln
    stock_price['Logarithmic_circulating_market_value'] = np.log(stock_price['total_mv'])

    stock_price = stock_price.reset_index(drop=True)
    if increment:
        stock_price = stock_price[stock_price['trade_date'] == trade_date]
    else:
        stock_price = stock_price[(stock_price['trade_date']  >= start_date) & (stock_price['trade_date']  <= trade_date)]
    stock_price = stock_price[['trade_date', 'ts_code','Logarithmic_circulating_market_value']]
    return stock_price

In [403]:
result = Logarithmic_circulating_market_value(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Logarithmic_circulating_market_value
601243,2024-08-14,000001.SZ,16.773058
601244,2024-08-14,000002.SZ,15.906018
...,...,...,...
601541,2024-08-14,688599.SH,15.121636
601542,2024-08-14,688981.SH,17.477702


0723流动性经营负债变动ok

In [1141]:
def Changes_in_current_operating_liabilities(trade_date, common_param, increment=True, start_date='20150101'):
    # 获取流动负债相关数据
    All_data = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'total_current_liability', 'shortterm_loan', 'tradable_fin_liab', 'notes_payable', 'current_assets_one_year']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 'total_current_liability', 'shortterm_loan', 'tradable_fin_liab', 'notes_payable', 'current_assets_one_year']]
        All_data = All_data.append(temp)
        All_data.fillna(0, inplace=True)
        # All_data = All_data.dropna()  # 去除空值的股票
    if increment:
        end_date = trade_date
    else:
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=15)).strftime("%Y%m%d")  # pct + 3
    All_data['m_timetag'] = pd.to_datetime(All_data['m_timetag'])
    All_data = All_data[(All_data['m_timetag'] >= new_start_date) & (All_data['m_timetag'] <= trade_date)]
    All_data['m_anntime'] = pd.to_datetime(All_data['m_anntime'])
    
    # 计算流动性经营负债
    All_data['Liquidity_operating_liabilities'] = All_data['total_current_liability'] - (
        All_data['shortterm_loan'] + 
        All_data['tradable_fin_liab'] + 
        All_data['notes_payable'] + 
        All_data['current_assets_one_year']
    )

    # 获取总资产数据
    All_data1 = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'tot_assets']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 'tot_assets']]
        All_data1 = All_data1.append(temp)
        # All_data = All_data.dropna()  # 去除空值的股票
    All_data1['m_anntime'] = pd.to_datetime(All_data1['m_anntime'])
    All_data1['m_timetag'] = pd.to_datetime(All_data1['m_timetag'])
    # 平均总资产（1年）
    All_data1['Avg'] = All_data1.groupby('ts_code')['tot_assets'].apply(lambda x: (x + x.shift(4)) / 2)

    # 内连接合并数据
    merged_df = pd.merge(All_data, All_data1, how='inner', left_on=['ts_code', 'm_anntime', 'm_timetag'], right_on=['ts_code', 'm_anntime', 'm_timetag'])

    # 计算流动性经营负债变动
    merged_df['Changes_in_current_operating_liabilities'] = (
        (merged_df['Liquidity_operating_liabilities'] - merged_df.groupby('ts_code')['Liquidity_operating_liabilities'].transform(lambda x:x.pct_change(4))) / merged_df['Avg']
    )
    
    Changes_in_current_operating_liabilities = merged_df[['m_timetag', 'ts_code', 'Changes_in_current_operating_liabilities']]
    Changes_in_current_operating_liabilities.columns = ['trade_date', 'ts_code', 'Changes_in_current_operating_liabilities']
    Changes_in_current_operating_liabilities = Changes_in_current_operating_liabilities.dropna()
    return Changes_in_current_operating_liabilities


In [955]:
result = Changes_in_current_operating_liabilities(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Changes_in_current_operating_liabilities
4,2021-03-31,000001.SZ,-0.011715
9,2021-03-31,000002.SZ,0.730810
...,...,...,...
1425,2021-03-31,688599.SH,0.258543
1430,2021-03-31,688981.SH,0.105254


1223成交额波动系数no1

In [967]:
start_date = '20150101' #设置全量计算时的开始时间
trade_date = '20240814' #每天的日期

In [1142]:
def Trading_volume_volatility_coefficient(trade_date, common_param, increment=True, start_date='20150101'):
    k = 3  # 定义最近k个月
    
    # 获取成交额数据
    df_st = common_param['df_amount_st']
    
    # 计算均值（mean）
    df_st_mean = df_st.rolling(21 * k).mean()  # 每月21个交易日，k个月
    
    # 计算标准差（std）
    df_st_std = df_st.rolling(21 * k).std()
    
    # 计算因子
    ans = df_st_mean / df_st_std
    
    ans = ans.stack().reset_index()
    ans.columns = ['trade_date', 'ts_code', 'Trading_volume_volatility_coefficient']
    ans = ans[['trade_date', 'ts_code', 'Trading_volume_volatility_coefficient']]
    Trading_volume_volatility_coefficient = ans.sort_values(by=['ts_code', 'trade_date'])
    Trading_volume_volatility_coefficient['trade_date'] = pd.to_datetime(Trading_volume_volatility_coefficient['trade_date'])
    if increment:
        Trading_volume_volatility_coefficient = Trading_volume_volatility_coefficient[Trading_volume_volatility_coefficient['trade_date'] == pd.to_datetime(trade_date)]
    else:
        Trading_volume_volatility_coefficient = Trading_volume_volatility_coefficient[(Trading_volume_volatility_coefficient['trade_date'] >= start_date) & (Trading_volume_volatility_coefficient['trade_date'] <= trade_date)]
    return Trading_volume_volatility_coefficient

In [972]:
result = Trading_volume_volatility_coefficient(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Trading_volume_volatility_coefficient
598270,2024-08-14,000001.SZ,2.079595
598271,2024-08-14,000002.SZ,1.110140
...,...,...,...
598568,2024-08-14,688599.SH,1.679514
598569,2024-08-14,688981.SH,2.072637


1212异常预收款ok

In [1143]:
def Abnormal_advance_payment(trade_date, common_param, increment=True, start_date='20150101'):
    # 获取预售款项和总资产数据
    All_data = pd.DataFrame()
    
    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'advance_peceipts', 'tot_assets']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 'advance_peceipts', 'tot_assets']]
        All_data = All_data.append(temp)
        # All_data = All_data.dropna()  # 去除空值的股票
    if increment:
        end_date = trade_date
    else:
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=15)).strftime("%Y%m%d")  # pct + 3
    All_data['m_timetag'] = pd.to_datetime(All_data['m_timetag'])
    All_data = All_data[(All_data['m_timetag'] >= new_start_date) & (All_data['m_timetag'] <= trade_date)]
    All_data['m_anntime'] = pd.to_datetime(All_data['m_anntime'])
    # 获取销售商品、提供劳务收到的现金数据
    All_data1 = pd.DataFrame()
    
    for asset in common_param['financial_data_ASHARECASHFLOW'].keys():
        temp = common_param['financial_data_ASHARECASHFLOW'][asset]['ASHARECASHFLOW'][['m_anntime', 'm_timetag', 'goods_sale_and_service_render_cash']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 'goods_sale_and_service_render_cash']]
        All_data1 = All_data1.append(temp)
        # All_data1 = All_data1.dropna()  # 去除空值的股票
    All_data1['m_anntime'] = pd.to_datetime(All_data1['m_anntime'])
    All_data1['m_timetag'] = pd.to_datetime(All_data1['m_timetag'])
    # 计算差异
    All_data1['Year'] = All_data1['m_timetag'].dt.year
    All_data1 = All_data1.groupby('ts_code').apply(lambda x: cal_factor_dif(x, 'goods_sale_and_service_render_cash'))

    # 计算正常增长乘数
    All_data1['Normal_growth_multiplier'] = All_data1['goods_sale_and_service_render_cash_diff'] / All_data1.groupby('ts_code')['goods_sale_and_service_render_cash_diff'].transform(lambda x:x.pct_change(4))
    
    # 合并数据
    merged_df = pd.merge(All_data, All_data1, how='inner', left_on=['ts_code', 'm_anntime', 'm_timetag'], right_on=['ts_code', 'm_anntime', 'm_timetag'])

    # 计算异常预收款
    merged_df['Abnormal_advance_payment'] = (merged_df['advance_peceipts'] - merged_df.groupby('ts_code')['advance_peceipts'].transform(lambda x:x.pct_change(4)) * merged_df['Normal_growth_multiplier']) / merged_df['tot_assets']
    
    Abnormal_advance_payment = merged_df[['m_timetag', 'ts_code', 'Abnormal_advance_payment']]
    Abnormal_advance_payment.columns = ['trade_date', 'ts_code', 'Abnormal_advance_payment']
    Abnormal_advance_payment = Abnormal_advance_payment.dropna()
    return Abnormal_advance_payment


In [456]:
result = Abnormal_advance_payment(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Abnormal_advance_payment
9,2024-06-30,000002.SZ,0.142476
14,2024-06-30,000063.SZ,0.305347
...,...,...,...
1490,2024-06-30,688599.SH,-0.725855
1495,2024-06-30,688981.SH,-0.012794


1209综合债务发行-年度ok

In [1144]:
def Logarithmic_growth_rate_of_total_liabilities(trade_date, common_param, increment=True, start_date='20150101'):
    # 获取总负债数据
    All_data = pd.DataFrame()

    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'tot_liab']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 'tot_liab']]
        All_data = All_data.append(temp)
        # All_data = All_data.dropna()  # 去除空值的股票
    if increment:
        end_date = trade_date
    else:
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=3)).strftime("%Y%m%d")  #3,4,5
    All_data['m_timetag'] = pd.to_datetime(All_data['m_timetag'])
    All_data = All_data[(All_data['m_timetag'] >= new_start_date) & (All_data['m_timetag'] <= trade_date)]
    All_data['m_anntime'] = pd.to_datetime(All_data['m_anntime'])
    
    # 获取去年总负债数据
    All_data_last = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'tot_liab']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 'tot_liab']]
        All_data_last = All_data_last.append(temp)
        # All_data_last = All_data_last.dropna()  # 去除空值的股票

    # 处理去年数据
    All_data_last['m_anntime'] = pd.to_datetime(All_data_last['m_anntime'])
    All_data_last['m_timetag'] = pd.to_datetime(All_data_last['m_timetag'])
    All_data_last['m_timetag'] = All_data_last['m_timetag'] - pd.DateOffset(years=1)
    All_data_last['tot_liab'] = All_data_last.groupby('ts_code')['tot_liab'].shift(4)

    # 合并当前年份和去年的数据
    merged_data = pd.merge(
        All_data,
        All_data_last,
        on=['ts_code', 'm_anntime'],
        suffixes=('', '_last_year')
    )

    # 计算总负债的对数增长率
    merged_data['Logarithmic_growth_rate_of_total_liabilities'] = (
        np.log(merged_data['tot_liab'] / merged_data['tot_liab_last_year'])
    )

    # 取近期的单季度数据
    merged_data = merged_data.drop_duplicates(subset=['ts_code', 'm_timetag'])
    # merged_data = merged_data.groupby('ts_code').apply(lambda x: x.tail(1))

    Logarithmic_growth_rate_of_total_liabilities = merged_data[['m_timetag', 'ts_code', 'Logarithmic_growth_rate_of_total_liabilities']]
    Logarithmic_growth_rate_of_total_liabilities.columns = ['trade_date', 'ts_code', 'Logarithmic_growth_rate_of_total_liabilities']
    Logarithmic_growth_rate_of_total_liabilities = Logarithmic_growth_rate_of_total_liabilities.dropna()
    #Logarithmic_growth_rate_of_total_liabilities=Logarithmic_growth_rate_of_total_liabilities.groupby('ts_code').tail(1)
    return Logarithmic_growth_rate_of_total_liabilities

In [1048]:
result = Logarithmic_growth_rate_of_total_liabilities(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Logarithmic_growth_rate_of_total_liabilities
0,2022-06-30,000001.SZ,0.077524
1,2022-06-30,000002.SZ,-0.067475
...,...,...,...
318,2022-06-30,688599.SH,0.416586
319,2022-06-30,688981.SH,0.373801


1205标准化的未预期盈余ok

In [1145]:
def Standardized_unexpected_earnings(trade_date, common_param, increment=True, start_date='20150101'):
    # 获取每股收益（基本）数据
    All_data = pd.DataFrame()

    for asset in common_param['financial_data_ASHAREINCOME'].keys():
        temp = common_param['financial_data_ASHAREINCOME'][asset]['ASHAREINCOME'][['m_anntime', 'm_timetag', 's_fa_eps_basic']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 's_fa_eps_basic']]
        All_data = All_data.append(temp)
        # All_data = All_data.dropna()  # 去除空值的股票
    if increment:
        end_date = trade_date
    else:
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=36)).strftime("%Y%m%d")  # 36
    All_data['m_timetag'] = pd.to_datetime(All_data['m_timetag'])
    All_data = All_data[(All_data['m_timetag'] >= new_start_date) & (All_data['m_timetag'] <= trade_date)]
    
    # 计算每股收益年度数据
    All_data['Year'] = All_data['m_timetag'].dt.year
    All_data = All_data.groupby('ts_code').apply(lambda x: cal_factor_dif(x, 's_fa_eps_basic'))

    # 计算盈余预差
    All_data['Earnings_forecast'] = All_data.groupby('ts_code')['s_fa_eps_basic_diff'].transform(lambda x: x - x.shift(4))
    All_data['Standardized_unexpected_earnings'] = All_data['Earnings_forecast']/All_data['Earnings_forecast'].rolling(8).std()

    Standardized_unexpected_earnings = All_data[['m_timetag', 'ts_code', 'Standardized_unexpected_earnings']]
    Standardized_unexpected_earnings.columns = ['trade_date', 'ts_code', 'Standardized_unexpected_earnings']
    Standardized_unexpected_earnings = Standardized_unexpected_earnings.dropna()
    return Standardized_unexpected_earnings

In [483]:
result = Standardized_unexpected_earnings(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Standardized_unexpected_earnings
109,2024-06-30,000001.SZ,0.080789
107,2024-06-30,000002.SZ,-2.745462
...,...,...,...
22,2024-06-30,688599.SH,-1.628737
23,2024-06-30,688981.SH,-0.192175


1118成交额波动no1

In [939]:
start_date = '20150101' #设置全量计算时的开始时间
trade_date = '20240814' #每天的日期

In [1146]:
def Trading_Volume_Fluctuation(trade_date, common_param, increment=True, start_date='20150101'):
    k = 3  # 定义最近k个月

    # 获取成交额数据
    df_st = common_param['df_amount_st']
    
    # 计算标准差（std）
    df_st_std = df_st.rolling(21 * k).std()
    
    df_st_std = df_st_std.stack().reset_index()
    df_st_std.columns = ['trade_date', 'ts_code', 'Trading_Volume_Fluctuation']
    df_st_std = df_st_std[['ts_code', 'trade_date', 'Trading_Volume_Fluctuation']]
    Trading_Volume_Fluctuation = df_st_std.sort_values(by=['trade_date','ts_code'])
    Trading_Volume_Fluctuation['trade_date'] = pd.to_datetime(Trading_Volume_Fluctuation['trade_date'])
    if increment:
        Trading_Volume_Fluctuation = Trading_Volume_Fluctuation[Trading_Volume_Fluctuation['trade_date'] == pd.to_datetime(trade_date)]
    else:
        Trading_Volume_Fluctuation = Trading_Volume_Fluctuation[(Trading_Volume_Fluctuation['trade_date'] >= start_date) & (Trading_Volume_Fluctuation['trade_date'] <= trade_date)]
    Trading_Volume_Fluctuation = Trading_Volume_Fluctuation[['trade_date','ts_code','Trading_Volume_Fluctuation']]
    return Trading_Volume_Fluctuation

In [986]:
result = Trading_Volume_Fluctuation(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Trading_Volume_Fluctuation
420600,2021-01-06,000001.SZ,6.847810e+08
420601,2021-01-06,000002.SZ,7.097305e+08
...,...,...,...
420898,2021-01-06,688599.SH,3.587438e+08
420899,2021-01-06,688981.SH,1.000591e+09


1112权益乘数ok

In [1147]:
def Equity_multiplier(trade_date, common_param, increment=False, start_date='20150101'):
    # 获取总资产数据
    All_data1 = pd.DataFrame()

    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'tot_assets']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 'tot_assets']]
        All_data1 = All_data1.append(temp)
        # All_data1 = All_data1.dropna()  # 去除空值的股票
        
    if increment:
        end_date = trade_date
    else:
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=6)).strftime("%Y%m%d")  # pct + 3
    All_data1['m_timetag'] = pd.to_datetime(All_data1['m_timetag'])
    All_data1 = All_data1[(All_data1['m_timetag'] >= new_start_date) & (All_data1['m_timetag'] <= trade_date)]
    All_data1['m_anntime'] = pd.to_datetime(All_data1['m_anntime'])
    
    # 获取归属于母公司股东权益合计数据
    All_data2 = pd.DataFrame()
    for asset in common_param['financial_data_ASHAREBALANCESHEET'].keys():
        temp = common_param['financial_data_ASHAREBALANCESHEET'][asset]['ASHAREBALANCESHEET'][['m_anntime', 'm_timetag', 'tot_shrhldr_eqy_excl_min_int']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 'tot_shrhldr_eqy_excl_min_int']]
        All_data2 = All_data2.append(temp)
        # All_data2 = All_data2.dropna()  # 去除空值的股票
    All_data2['m_anntime'] = pd.to_datetime(All_data2['m_anntime'])
    All_data2['m_timetag'] = pd.to_datetime(All_data2['m_timetag'])
    # 计算平均值
    All_data1['Avg1'] = All_data1.groupby('ts_code')['tot_assets'].apply(lambda x: (x + x.shift(1)) / 2)
    All_data2['Avg2'] = All_data2.groupby('ts_code')['tot_shrhldr_eqy_excl_min_int'].apply(lambda x: (x + x.shift(1)) / 2)

    # 合并数据并计算权益乘数
    merged_df = pd.merge(All_data1, All_data2, how='inner', left_on=['ts_code', 'm_anntime', 'm_timetag'], right_on=['ts_code', 'm_anntime', 'm_timetag'])
    merged_df['Equity_multiplier'] = merged_df['Avg1'] / merged_df['Avg2']

    Equity_multiplier = merged_df[['m_timetag', 'ts_code', 'Equity_multiplier']]
    Equity_multiplier.columns = ['trade_date', 'ts_code', 'Equity_multiplier']
    Equity_multiplier = Equity_multiplier.dropna()
    return Equity_multiplier 

In [549]:
result = Equity_multiplier(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Equity_multiplier
1,2021-12-31,000001.SZ,12.462552
3,2021-12-31,000002.SZ,8.425892
...,...,...,...
590,2021-12-31,688599.SH,3.743475
592,2021-12-31,688981.SH,2.061155


1108单季度经营性活动产生的现金流净额同比增长率ok

In [1148]:
def Net_cash_flow_from_operating_activities(trade_date, common_param, increment=True, start_date='20150101'):
    # 获取单季度经营性活动产生的现金流净额数据
    All_data = pd.DataFrame()

    for asset in common_param['financial_data_ASHARECASHFLOW'].keys():
        temp = common_param['financial_data_ASHARECASHFLOW'][asset]['ASHARECASHFLOW'][['m_anntime', 'm_timetag', 'net_cash_flows_oper_act']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 'net_cash_flows_oper_act']]
        All_data = All_data.append(temp)
        # All_data = All_data.dropna()  # 去除空值的股票
    if increment:
        end_date = trade_date
    else:
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=24)).strftime("%Y%m%d")  # pct + 3
    All_data['m_timetag'] = pd.to_datetime(All_data['m_timetag'])
    All_data = All_data[(All_data['m_timetag'] >= new_start_date) & (All_data['m_timetag'] <= trade_date)]
    # 提取年份和月份信息
    All_data['Year'] = All_data['m_timetag'].dt.year
    All_data['Month'] = All_data['m_timetag'].dt.month

    # 计算最近的TTM（Trailing Twelve Months，最近12个月的数据）
    All_data = All_data.groupby('ts_code').apply(lambda x: cal_factor_ttm(x, 'net_cash_flows_oper_act'))
    
    #去年
    merged_data = All_data ####？goupby('ts_code')['net_cash_flows_oper_act_TTM'].transform(lambda x:x.pct_change(4))
    merged_data['Net_cash_flow_from_operating_activities'] = (
        (merged_data['net_cash_flows_oper_act_TTM'] - merged_data.groupby('ts_code')['net_cash_flows_oper_act_TTM'].transform(lambda x:x.pct_change(4))) /abs(merged_data.groupby('ts_code')['net_cash_flows_oper_act_TTM'].transform(lambda x:x.pct_change(4)))
    )

    Net_cash_flow_from_operating_activities = merged_data[['m_timetag', 'ts_code', 'Net_cash_flow_from_operating_activities']]
    Net_cash_flow_from_operating_activities.columns = ['trade_date', 'ts_code', 'Net_cash_flow_from_operating_activities']
    Net_cash_flow_from_operating_activities = Net_cash_flow_from_operating_activities.dropna()
    return Net_cash_flow_from_operating_activities

In [578]:
result = Net_cash_flow_from_operating_activities(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Net_cash_flow_from_operating_activities
95,2024-06-30,000001.SZ,1.716397e+12
94,2024-06-30,000002.SZ,-1.864527e+09
...,...,...,...
22,2024-06-30,688599.SH,2.816565e+10
23,2024-06-30,688981.SH,2.264861e+10


1015市盈率增长率ok

In [1149]:
def Price_to_earnings_ratio_growth_rate(trade_date, common_param, increment=True, start_date='20150101'):
    # 获取市盈率TTM数据
    stock_price = common_param['pe_ttm']

    # 获取12月归属母公司净利润数据
    All_data = pd.DataFrame()

    for asset in common_param['financial_data_ASHAREINCOME'].keys():
        temp = common_param['financial_data_ASHAREINCOME'][asset]['ASHAREINCOME'][['m_anntime', 'm_timetag', 'net_profit_excl_min_int_inc']]
        temp['ts_code'] = asset
        temp = temp[['ts_code', 'm_anntime', 'm_timetag', 'net_profit_excl_min_int_inc']]
        All_data = All_data.append(temp)
    if increment:
        end_date = trade_date
    else:
        end_date = start_date
    new_start_date = (pd.to_datetime(end_date) - pd.DateOffset(months=28)).strftime("%Y%m%d")  # pct + 3
    All_data['m_timetag'] = pd.to_datetime(All_data['m_timetag'])
    All_data = All_data[(All_data['m_timetag'] >= new_start_date) & (All_data['m_timetag'] <= trade_date)]
    
    All_data['Year'] = All_data['m_timetag'].dt.year
    All_data['Month'] = All_data['m_timetag'].dt.month

    # 计算最近的TTM
    All_data = All_data.groupby('ts_code').apply(lambda x: cal_factor_ttm(x, 'net_profit_excl_min_int_inc'))

    # 计算12个月归属母公司同比增速TTM
    All_data['profit_TTM'] = (
        (All_data['net_profit_excl_min_int_inc_TTM'] - 
         All_data.groupby('ts_code')['net_profit_excl_min_int_inc_TTM'].transform(lambda x: x.pct_change(4)))
        / All_data.groupby('ts_code')['net_profit_excl_min_int_inc_TTM'].transform(lambda x: x.pct_change(4))
    )
    #.groupby('ts_code')['net_profit_excl_min_int_inc_TTM'].transform(lambda x:x.pct_change(4)
    # 合并市盈率和净利润数据
    All_data['m_anntime'] = pd.to_datetime(All_data['m_anntime'])
    ans_factor = pd.merge(All_data, stock_price, how='inner', left_on=['ts_code', 'm_anntime'], right_on=['ts_code', 'trade_date'])

    # 计算市盈率增长率因子
    ans_factor['Price_to_earnings_ratio_growth_rate'] = ans_factor['pe_ttm'] / ans_factor['profit_TTM']

    Price_to_earnings_ratio_growth_rate = ans_factor[['m_timetag', 'ts_code', 'Price_to_earnings_ratio_growth_rate']]
    Price_to_earnings_ratio_growth_rate.columns = ['trade_date', 'ts_code', 'Price_to_earnings_ratio_growth_rate']
    Price_to_earnings_ratio_growth_rate = Price_to_earnings_ratio_growth_rate.dropna()
    return Price_to_earnings_ratio_growth_rate

In [603]:
result = Price_to_earnings_ratio_growth_rate(trade_date, common_param, increment=True, start_date='20150101')
result

,trade_date,ts_code,Price_to_earnings_ratio_growth_rate
11,2024-03-31,000002.SZ,-4.697048e-10
14,2024-03-31,000063.SZ,-1.873731e-10
...,...,...,...
1755,2024-03-31,688599.SH,-5.254418e-10
1761,2024-03-31,688981.SH,-1.778059e-08


In [729]:
start_date = '20150101' #设置全量计算时的开始时间
trade_date = '20240814' #每天的日期

In [1168]:
# 将所有的因子函数集中在一个dict中
fact_dict = {
    "mean_amount": mean_amount,#1025成交额ok
    "TOP10stocks": TOP10stocks,#1022十大股东占比分散度okk
    "inventory_growth_rate": inventory_growth_rate,#1019存货增长率ok
    "Unexpected_tax_expenses": Unexpected_tax_expenses,#1010未预期税费支出ok
    "Net_profit_margin_change": Net_profit_margin_change,#1001单季度总资产净利率变动ok 
    #"Market_leverage": Market_leverage,#0926市场杠杆no1
    "Momentum_based_on_amplitude_cutting": Momentum_based_on_amplitude_cutting,#0922基于振幅切割的动量ok
    "Return_on_total_assets_change": Return_on_total_assets_change,#0902单季度总资产报酬率变动ok
    "Improvement_of_business_performance": Improvement_of_business_performance,#0828营业能力改善ok
    "Profit_to_market_ratio": Profit_to_market_ratio,#0826盈利市值比ok 
    "book_to_market": book_to_market,#0822账面市值比okk
    #"Intraday_rate_of": Intraday_rate_of,#0805日内收益率no1
    #"Logarithmic_circulating_market_value": Logarithmic_circulating_market_value,#0729对数流通市值 no1
    "Changes_in_current_operating_liabilities": Changes_in_current_operating_liabilities,#0723流动性经营负债变动ok
    #"Trading_volume_volatility_coefficient": Trading_volume_volatility_coefficient,#1223成交额波动系数no1
    "Abnormal_advance_payment": Abnormal_advance_payment,#1212异常预收款ok
    "Logarithmic_growth_rate_of_total_liabilities": Logarithmic_growth_rate_of_total_liabilities,#1209综合债务发行-年度ok
    "Standardized_unexpected_earnings": Standardized_unexpected_earnings,#1205标准化的未预期盈余ok
    #"Trading_Volume_Fluctuation": Trading_Volume_Fluctuation,#1118成交额波动no1
    "Equity_multiplier": Equity_multiplier,#1112权益乘数ok
    "Net_cash_flow_from_operating_activities": Net_cash_flow_from_operating_activities,#1108单季度经营性活动产生的现金流净额同比增长率ok
    "Price_to_earnings_ratio_growth_rate": Price_to_earnings_ratio_growth_rate,#1015市盈率增长率ok
}